In [5]:
#with open(r"news_0_300_2.bin", "rb") as f:
#    vectores = f.read()
        

In [28]:
with open('ruscorpora_upos_skipgram_300_5_2018.vec') as f: #НКРЯ
    lines = f.readlines()

In [29]:
len(lines)

195072

In [30]:
lines[10]

'самый_ADJ 0.025079 0.009604 0.013054 0.091902 0.023336 -0.011569 0.021876 -0.021861 -0.126836 0.010479 -0.015957 -0.082398 0.024110 0.036226 0.055787 -0.045762 0.127345 -0.044893 0.059016 -0.040686 0.055058 -0.048654 -0.101892 -0.006459 -0.014031 -0.099471 0.041854 -0.141021 -0.085141 -0.065855 -0.039711 0.014697 -0.044900 0.037260 -0.108848 0.112243 0.018223 -0.086836 0.024476 0.115713 0.056492 0.064191 -0.003348 0.069250 0.049200 -0.076793 0.090253 -0.062914 0.020435 -0.078086 -0.017944 0.104103 0.027111 -0.058028 0.001862 0.010622 0.027744 -0.008368 -0.042689 -0.016448 -0.045289 0.008422 -0.060038 0.065562 -0.009644 -0.153754 0.129936 -0.064181 -0.030110 0.132179 -0.143709 -0.072943 0.014436 0.004676 0.025420 -0.064474 0.015054 0.015370 0.043522 0.033964 0.098297 0.127453 -0.007863 -0.023444 0.039410 -0.029944 -0.017899 -0.010531 -0.054708 0.042787 0.030037 0.043100 -0.122886 -0.077032 0.068674 -0.035136 -0.052920 -0.003404 0.001584 -0.083801 0.085360 0.108487 0.028873 0.105646 0.0

In [3]:
#words = [lines[i].split()[0].split('_')[0] for i in range(1, len(lines))]

In [31]:
words = [lines[i].split()[0] for i in range(1, len(lines))]

In [32]:
from collections import Counter
Counter(words).most_common(100)

[('мочь_VERB', 1),
 ('год_NOUN', 1),
 ('xx_NUM', 1),
 ('человек_NOUN', 1),
 ('сказать_VERB', 1),
 ('говорить_VERB', 1),
 ('время_NOUN', 1),
 ('xxxx_NUM', 1),
 ('знать_VERB', 1),
 ('самый_ADJ', 1),
 ('дело_NOUN', 1),
 ('делать_VERB', 1),
 ('день_NOUN', 1),
 ('жизнь_NOUN', 1),
 ('видеть_VERB', 1),
 ('рука_NOUN', 1),
 ('очень_ADV', 1),
 ('первый_ADJ', 1),
 ('новый_ADJ', 1),
 ('давать_VERB', 1),
 ('становиться_VERB', 1),
 ('слово_NOUN', 1),
 ('должен_ADJ', 1),
 ('иметь_VERB', 1),
 ('идти_VERB', 1),
 ('xxx_NUM', 1),
 ('большой_ADJ', 1),
 ('глаз_NOUN', 1),
 ('лицо_NOUN', 1),
 ('хотеть_VERB', 1),
 ('место_NOUN', 1),
 ('понимать_VERB', 1),
 ('смотреть_VERB', 1),
 ('каждый_ADJ', 1),
 ('работа_NOUN', 1),
 ('казаться_VERB', 1),
 ('писать_VERB', 1),
 ('голова_NOUN', 1),
 ('оставаться_VERB', 1),
 ('сторона_NOUN', 1),
 ('друг_NOUN', 1),
 ('спрашивать_VERB', 1),
 ('думать_VERB', 1),
 ('брать_VERB', 1),
 ('начинать_VERB', 1),
 ('жить_VERB', 1),
 ('дом_NOUN', 1),
 ('приходить_VERB', 1),
 ('последний_AD

In [33]:
'мужчина_NOUN' in words

True

In [34]:
'женщина_NOUN' in words

True

In [35]:
'художник-мультипликатор_NOUN' in words

False

In [36]:
import pandas as pd

df_prof = pd.read_csv('./professions.csv')

In [15]:
df_prof.columns

Index(['профессии'], dtype='object')

In [6]:
df_prof['профессии'][0]

'авербандщик'

In [37]:
df_prof['профессии'][0] + '_NOUN'

'авербандщик_NOUN'

In [38]:
search_list = []
for word in words:
    for prof in set(df_prof['профессии']):
        if word == prof + '_NOUN':
            search_list.append(prof)
        if len(search_list) == 300:       #когда уже набрали 300, то останавлимаемся
            break

In [39]:
search_list    #список из 300 профессий, нам надо отобрать, какие хотим оставить

['начальник',
 'сестра',
 'представитель',
 'глава',
 'художник',
 'врач',
 'министр',
 'командир',
 'директор',
 'капитан',
 'профессор',
 'учитель',
 'президент',
 'специалист',
 'работник',
 'председатель',
 'руководитель',
 'сотрудник',
 'мастер',
 'инженер',
 'артист',
 'секретарь',
 'губернатор',
 'судья',
 'корреспондент',
 'помощник',
 'режиссер',
 'заместитель',
 'редактор',
 'матрос',
 'охотник',
 'посол',
 'следователь',
 'прокурор',
 'кандидат',
 'боец',
 'агент',
 'советник',
 'водитель',
 'адвокат',
 'летчик',
 'эксперт',
 'комиссар',
 'сторож',
 'преподаватель',
 'производитель',
 'исполнитель',
 'переводчик',
 'старшина',
 'дворник',
 'проводник',
 'строитель',
 'архитектор',
 'инспектор',
 'оператор',
 'педагог',
 'организатор',
 'комендант',
 'физик',
 'продавец',
 'разведчик',
 'кучер',
 'рыбак',
 'мэр',
 'наблюдатель',
 'пристав',
 'химик',
 'конструктор',
 'повар',
 'посланник',
 'охранник',
 'тренер',
 'пилот',
 'психолог',
 'няня',
 'юрист',
 'смотритель',
 'бриг